Initial loading sequence. Only need to initialize this one time

In [2]:
!pip install transformers

     |████████████████████████████████| 890kB 2.8MB/s 
     |████████████████████████████████| 3.0MB 15.3MB/s 
     |████████████████████████████████| 890kB 27.1MB/s 
     |████████████████████████████████| 1.1MB 11.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=7e8b2e9119a9a253ad86d2c27d919e059f1fb3075cbfc58c141df907f23b3595
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
#importing relevant libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB

import torch

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

from keras.models import load_model


loadedModel = load_model('/content/bertmodelRegLabels.h5')



Data testing. If the printed number is less than 0.5, they're depressed, and if not they're suicidal. 

In [12]:
test_data = {0: ["I've always been trashed my whole life. When people don't see a reason to use me anymore, I get dumped, this shit happens since my first teenage years and idk why at all. I try to always be kind with everyone, I apologize for my mistakes, I try to help people around me but when I need no one fucking helps me. I'm tired of being alone, I'm tired of always being the third wheel, it just seems like everyone else is better and more important than me, i just want someone to actually care about me."]}
test_text = pd.DataFrame(test_data, columns = [0])

## Want BERT instead of distilBERT? Uncomment the following line:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

tokenized = test_text[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=128)))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

features = last_hidden_states[0][:,0,:].numpy()


predictions = loadedModel.predict(features)
print(predictions)

[[0.18759483]]
